In [1]:
# pdf plumber
import requests
from io import BytesIO
import pdfplumber

# data analysis
import numpy as np
import pandas as pd
import re
import csv

import scraper

In [2]:
tsmc_urls_file = 'urls/tsmc_urls.csv'
tsmc_urls_reader = csv.reader(open( 'urls/tsmc_urls.csv', 'r', encoding='utf-8-sig'))
tsmc_urls = dict([(row[0], row[1]) for row in tsmc_urls_reader])

In [3]:
for quarter, url in tsmc_urls.items():
    print(quarter, url)
    if url not in ('NA', 'Link'):
        scraper.parse_tsmc(url)

1Q11 https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q1/1Q11%20Management%20Report.pdf
Capex failed for https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q1/1Q11%20Management%20Report.pdf
Inv failed for https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q1/1Q11%20Management%20Report.pdf
2Q11 https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q2/2Q11%20Management%20Report.pdf
URL not working for https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q2/2Q11%20Management%20Report.pdf!
3Q11 https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q3/3Q11%20Management%20Report.pdf
URL not working for https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q3/3Q11%20Management%20Report.pdf!
4Q11 https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q4/4Q11%20Management%20Report.pdf
URL not working for https://investor.tsmc.

In [ ]:
url = "https://investor.tsmc.com/english/encrypt/files/encrypt_file/english/2011/Q1/1Q11%20Management%20Report.pdf"
rq = requests.get(url)
pdf = pdfplumber.open(BytesIO(rq.content))
tsmc_text = ""
for i in range(5):
    tsmc_text += pdf.pages[i].extract_text()

In [23]:
tsmc_text

'FOR IMMEDIATE RELEASE  \n1Q11  Quarterly Management Report\nApril 28, 2011 \n       Operating Results Review: \n \nCONTACT \nE lizabeth Sun    S ummary:\nC orporate Communication Division   \nTSMC \nin vest@tsmc.com  (A mounts are on consolidated basis and are in NT\n88 6-3-568-2085    b illions except otherwise noted) 1Q11 4Q10 1Q10 QoQ YoY\n         \n   \nEPS (NT$ per common share) 1.40 1.57 1.30 (10.9%) 7.7%\n           (US$ per ADR unit) 0.24 0.26 0.20\n \n \nConsolidated Net Sales 105.38 110.14 92.19 (4.3%) 14.3%\n Topics in This Report \n \nGross Profit    51.67 54.82 44.19 (5.7%) 16.9%\n  \nG ross Margin 49.0% 49.8% 47.9%\n •  Rev enue An alysis               \n \n   Operating Expense (12.51) (13.31) (10.06) (6.0%) 24.4%\n •  Profit & Expense  O perating Income 39.16 41.51 34.13 (5.7%) 14.8%\nO perating Margin 37.2% 37.7% 37.0%\n  Analysis \n     N on-Operating Items 1.15 1.37 1.18\n •  Financial  N et Income 36.28 40.72 33.66 (10.9%) 7.8%\n  Condition Review  N et Profit Marg

In [24]:
def clean_tsmc_geo(tsmc_text):
  """
  Cleans TSMC's geo dataframe.
  """
  umcRegions = []
  digits = "\s+(\d+%)\s+(\d+%)\s+(\d+%)"
  regionCols = np.asarray(re.findall(r"(Net Revenue by Geography|By Geography)\s+(\d+Q\d+)\s+(\d+Q\d+)\s+(\d+Q\d+)", tsmc_text)[0])
  NorthAm = np.asarray(re.findall(f"(North America){digits}", tsmc_text)[0])
  umcRegions.append(NorthAm)
  China = np.asarray(re.findall(f"(China){digits}", tsmc_text)[0])
  umcRegions.append(China)
  try:
      EMEA = np.asarray(re.findall(f"(EMEA){digits}", tsmc_text)[0])
      umcRegions.append(EMEA)
  except IndexError:
      pass
  try:
      europe = np.asarray(re.findall(f"(Europe){digits}", tsmc_text)[0])
      umcRegions.append(europe)
  except IndexError:
      pass
  try: 
      APAC = np.asarray(re.findall(f"(Asia Pacific){digits}", tsmc_text)[0])
      umcRegions.append(APAC)
  except IndexError:
      pass
  Japan = np.asarray(re.findall(f"(Japan){digits}", tsmc_text)[0])
  umcRegions.append(Japan)
  geoDF = pd.DataFrame(umcRegions, columns=regionCols)
  return geoDF

In [25]:
clean_tsmc_geo(tsmc_text)

,By Geography,1Q11,4Q10,1Q10
0,North America,69%,70%,68%
1,China,3%,3%,2%
2,Europe,10%,10%,11%
3,Asia Pacific,14%,13%,15%
4,Japan,4%,4%,4%
